In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score,recall_score, confusion_matrix,  roc_auc_score
import numpy as np
import time
import pickle
"Own made libraries"
import database as db
import scrapping.scrapperSelenium as scrapperSelenium
import scrapping.scrapperSoup as scrapperSoup
import textCleaning as tc
import model.buildmodel as build
import scrapping.scrapperSelenium as selenium
import scrapping.scrapperSoup as soup

[nltk_data] Downloading package wordnet to C:\Users\Roy
[nltk_data]     Dijkstra\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Roy
[nltk_data]     Dijkstra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Roy
[nltk_data]     Dijkstra\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
link = "https://www.tripadvisor.com/Hotel_Review-g14129477-d10426242-Reviews-HOSHINOYA_Tokyo-Otemachi_Chiyoda_Tokyo_Tokyo_Prefecture_Kanto.html"
scrappedReviews = selenium.selenium_scrapper_tripadvisor(link, 20)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Roy Dijkstra\.wdm\drivers\chromedriver\win32\99.0.4844.51]


Scrapping done, adding labels


In [12]:
df = soup.soup_scrapper_booking("https://www.booking.com/reviews/nl/hotel/westcord-city-centre.en-gb.html", num_of_pages=10)

In [3]:
dfReviewsOwn = pd.read_csv('data/own_reviews.csv')
dfReviewsOwn = tc.own_label_data(dfReviewsOwn)
dfReviewsOwn = tc.clean_data(dfReviewsOwn)

In [2]:
dfTrip = pd.read_csv('data/trip_reviews.csv')
dfBooking = pd.read_csv('data/booking_reviews.csv')
dfHotels = pd.concat([dfBooking, dfTrip], ignore_index=True)
dfHotels = tc.clean_hotel_data(dfHotels)

In [4]:
dfKaggle = pd.read_csv("data/Hotel_Reviews.csv")
dfKaggle = tc.kaggle_strip_reviews(dfKaggle)
dfKaggle = tc.kaggle_label_data(dfKaggle)
dfKaggle = tc.clean_data(dfKaggle)


E:\Roy Dijkstra\School\BigData\BDSE\Individual_Assignment_1\textCleaning.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfStrippedReviews["Review"] = dfReviews["Negative_Review"].apply(lambda x: x.replace("No Negative", "")) + \


In [5]:
dfCombined = pd.concat([dfKaggle, dfHotels, dfReviewsOwn], ignore_index=True)

In [6]:
db.insert_df_into_db(dfCombined, 'cleanedhoteldata', 'replace')

Inserting data into database per 10000 rows


In [48]:
X = dfCombined['lemReviews']
y = dfCombined['label']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=69)

vect = CountVectorizer(max_features=1000)
X_train_vect = vect.fit_transform(X_train)
X_test_vect = vect.transform(X_test)

In [40]:
model = pickle.load(open('model/saved/finalized_lr_model_best_params2.sav', 'rb'))

In [49]:
vect = pickle.load(open('model/saved/lrvect.sav', 'rb'))

X_test_vect = vect.transform(X)
y_pred = model.predict(X_test_vect)
y_prob = model.predict_proba(X_test_vect)[::, 1]
print("Accuracy: {:.2f}%".format(accuracy_score(y, y_pred) * 100))
print("Confusion Matrix:\n", confusion_matrix(y, y_pred))
print("AUC: \n", roc_auc_score(y, y_prob))
print("Precision: \n", precision_score(y, y_pred, average='weighted'))
print("Recall: \n", recall_score(y, y_pred, average='weighted'))

Accuracy: 98.18%
Confusion Matrix:
 [[ 33553   6939]
 [  2469 473233]]
AUC: 
 0.9812581689297968
Precision: 
 0.9813058966645906
Recall: 
 0.9817742941607225
